In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import beta
import numpy as np
class Policy_Network(nn.Module):
    def __init__(self, input_size, action_size):
        super(Policy_Network, self).__init__()
        # outputs beta likelyhood parameters
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2*action_size)
        self.fcalpha = nn.Linear(2*action_size, action_size)
        self.fcbeta = nn.Linear(2*action_size, action_size)
        self.softplus_act = nn.Softplus()
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        a = self.softplus_act(self.fcalpha(x)) + 1.
        b = self.softplus_act(self.fcbeta(x)) + 1.
        beta_dist = beta.Beta(a,b)
        return beta_dist, x

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import beta
import numpy as np
class Policy_Value_Network(nn.Module):
    def __init__(self, input_size, action_size):
        super(Policy_Value_Network, self).__init__()
        # outputs beta likelyhood parameters
        self.policy_network = Policy_Network(input_size, action_size)
        self.fcval = nn.Linear(2*action_size, 1)
        
    def forward(self, x):
        beta_dist,x = self.policy_network(x)
        values = self.fcval(x)
        return beta_dist, values

In [4]:
import os, sys
sys.path.append("/Users/n0k03zp/rl/")

In [5]:
from policy_gradient_methods.a2c import A2C_PPO_LOSS
from policy_gradient_methods.utils import *
from example_environment_classes.gym_mujoco__parallel_env_class import MujocoGymNumStateContActions
from itertools import chain
import torch.optim as optim
learning_rate = 1e-3
episode = 50000
tmax = 1000
state_dim = 23
action_dim = 7
envs = MujocoGymNumStateContActions("Pusher-v4")
# Initializing policy network, optimizer and agent
device = get_device()
policy_value_network = Policy_Value_Network(state_dim,action_dim).to(device)
optimizer = optim.Adam(policy_value_network.parameters(), lr=learning_rate)
a2c_agent = A2C_PPO_LOSS(policy_value_network,
                 optimizer,
                 value_loss_coef=1,
                 n_boot_strap=1,
                 normalize_advantage =True,
                 use_gae_advantage = True,
                 lambda_bootstrap_schedule = lambda old_value, rewards: old_value,
                 lambda_bootstrap_start=0.8,
                 ppo_value_epsilon_schedule = lambda old_value, rewards: max([0.99995*old_value, 0.5]),
                 ppo_value_epsilon_start = 5,
                 SGD_steps=4,
                 ppo_policy_epsilon_schedule=lambda old_value, rewards: max([0.99995*old_value, 0.09]),
                 ppo_policy_epsilon_start=0.2,
                 entropy_reg_schedule = lambda old_value, rewards: 0.99995*old_value,
                 entropy_reg_start=0.01,
                 gamma_schedule = lambda old_value, rewards: 1.- 0.99995*(1. - old_value),
                 gamma_start = 0.98,
                 target_score=30,
                 target_score_window=100,
                 action_transform = lambda x: -2+4*x,
                 verbosity = 100)
a2c_agent.training(envs, tmax, tmax, episode)

action space: Box(-2.0, 2.0, (20, 7), float32)
raw state space : Box(-inf, inf, (20, 23), float64)
inside a2c
 Episode : 100	 Average reward in last 100 episode : -791.20 epsilon_policy : 0.199 epsilon_value : 4.975 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 200	 Average reward in last 100 episode : -576.15 epsilon_policy : 0.198 epsilon_value : 4.950 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 300	 Average reward in last 100 episode : -523.79 epsilon_policy : 0.197 epsilon_value : 4.926 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 400	 Average reward in last 100 episode : -509.53 epsilon_policy : 0.196 epsilon_value : 4.901 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 500	 Average reward in last 100 episode : -497.94 epsilon_policy : 0.195 epsilon_value : 4.877 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 600	 Average reward in last 100 episode : -488.84 epsilon_policy : 0.194 epsilon_val

/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/glfw/__init__.py:912: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
Exception ignored in: <function WindowViewer.__del__ at 0x2905611b0>
Traceback (most recent call last):
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 335, in __del__
    self.free()
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 330, in free
    glfw.destroy_window(self.window)
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/glfw/__init__.py", line 1275, in destroy_window
    window_addr = ctypes.cast(ctypes.pointer(window),
TypeError: _type_ must have storage info


 Episode : 1100	 Average reward in last 100 episode : -436.81 epsilon_policy : 0.189 epsilon_value : 4.732 lambda_gae : 0.800 gamma : 0.981 entropy_reg : 0.009
 Episode : 1200	 Average reward in last 100 episode : -426.54 epsilon_policy : 0.188 epsilon_value : 4.709 lambda_gae : 0.800 gamma : 0.981 entropy_reg : 0.009
 Episode : 1300	 Average reward in last 100 episode : -420.57 epsilon_policy : 0.187 epsilon_value : 4.685 lambda_gae : 0.800 gamma : 0.981 entropy_reg : 0.009
 Episode : 1400	 Average reward in last 100 episode : -418.58 epsilon_policy : 0.186 epsilon_value : 4.662 lambda_gae : 0.800 gamma : 0.981 entropy_reg : 0.009
 Episode : 1500	 Average reward in last 100 episode : -416.88 epsilon_policy : 0.186 epsilon_value : 4.639 lambda_gae : 0.800 gamma : 0.981 entropy_reg : 0.009
 Episode : 1600	 Average reward in last 100 episode : -412.67 epsilon_policy : 0.185 epsilon_value : 4.616 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 1700	 Average reward in last 

Exception ignored in: <function WindowViewer.__del__ at 0x2905611b0>
Traceback (most recent call last):
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 335, in __del__
    self.free()
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 330, in free
    glfw.destroy_window(self.window)
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/glfw/__init__.py", line 1275, in destroy_window
    window_addr = ctypes.cast(ctypes.pointer(window),
TypeError: _type_ must have storage info


 Episode : 2100	 Average reward in last 100 episode : -401.42 epsilon_policy : 0.180 epsilon_value : 4.502 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 2200	 Average reward in last 100 episode : -400.88 epsilon_policy : 0.179 epsilon_value : 4.479 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 2300	 Average reward in last 100 episode : -400.50 epsilon_policy : 0.178 epsilon_value : 4.457 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 2400	 Average reward in last 100 episode : -398.07 epsilon_policy : 0.177 epsilon_value : 4.435 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 2500	 Average reward in last 100 episode : -398.39 epsilon_policy : 0.176 epsilon_value : 4.412 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 2600	 Average reward in last 100 episode : -396.22 epsilon_policy : 0.176 epsilon_value : 4.390 lambda_gae : 0.800 gamma : 0.982 entropy_reg : 0.009
 Episode : 2700	 Average reward in last 

...

Exception ignored in: <function WindowViewer.__del__ at 0x2905611b0>
Traceback (most recent call last):
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 335, in __del__
    self.free()
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 330, in free
    glfw.destroy_window(self.window)
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/glfw/__init__.py", line 1275, in destroy_window
    window_addr = ctypes.cast(ctypes.pointer(window),
TypeError: _type_ must have storage info


 Episode : 12100	 Average reward in last 100 episode : -250.27 epsilon_policy : 0.109 epsilon_value : 2.730 lambda_gae : 0.800 gamma : 0.989 entropy_reg : 0.005
 Episode : 12200	 Average reward in last 100 episode : -249.95 epsilon_policy : 0.109 epsilon_value : 2.717 lambda_gae : 0.800 gamma : 0.989 entropy_reg : 0.005
 Episode : 12300	 Average reward in last 100 episode : -249.96 epsilon_policy : 0.108 epsilon_value : 2.703 lambda_gae : 0.800 gamma : 0.989 entropy_reg : 0.005
 Episode : 12400	 Average reward in last 100 episode : -248.51 epsilon_policy : 0.108 epsilon_value : 2.690 lambda_gae : 0.800 gamma : 0.989 entropy_reg : 0.005
 Episode : 12500	 Average reward in last 100 episode : -248.55 epsilon_policy : 0.107 epsilon_value : 2.676 lambda_gae : 0.800 gamma : 0.989 entropy_reg : 0.005
 Episode : 12600	 Average reward in last 100 episode : -247.42 epsilon_policy : 0.107 epsilon_value : 2.663 lambda_gae : 0.800 gamma : 0.989 entropy_reg : 0.005
 Episode : 12700	 Average reward i

Exception ignored in: <function WindowViewer.__del__ at 0x2905611b0>
Traceback (most recent call last):
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 335, in __del__
    self.free()
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 330, in free
    glfw.destroy_window(self.window)
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/glfw/__init__.py", line 1275, in destroy_window
    window_addr = ctypes.cast(ctypes.pointer(window),
TypeError: _type_ must have storage info


 Episode : 13100	 Average reward in last 100 episode : -247.73 epsilon_policy : 0.104 epsilon_value : 2.597 lambda_gae : 0.800 gamma : 0.990 entropy_reg : 0.005
 Episode : 13200	 Average reward in last 100 episode : -252.49 epsilon_policy : 0.103 epsilon_value : 2.584 lambda_gae : 0.800 gamma : 0.990 entropy_reg : 0.005
 Episode : 13300	 Average reward in last 100 episode : -264.94 epsilon_policy : 0.103 epsilon_value : 2.571 lambda_gae : 0.800 gamma : 0.990 entropy_reg : 0.005
 Episode : 13400	 Average reward in last 100 episode : -258.09 epsilon_policy : 0.102 epsilon_value : 2.559 lambda_gae : 0.800 gamma : 0.990 entropy_reg : 0.005
 Episode : 13500	 Average reward in last 100 episode : -255.20 epsilon_policy : 0.102 epsilon_value : 2.546 lambda_gae : 0.800 gamma : 0.990 entropy_reg : 0.005
 Episode : 13600	 Average reward in last 100 episode : -253.55 epsilon_policy : 0.101 epsilon_value : 2.533 lambda_gae : 0.800 gamma : 0.990 entropy_reg : 0.005
 Episode : 13700	 Average reward i

KeyboardInterrupt: 

In [18]:
envs.play(a2c_agent, 1000)

 score @ 753 = -0.33993789629339337

Exception ignored in: <function WindowViewer.__del__ at 0x2905611b0>
Traceback (most recent call last):
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 335, in __del__
    self.free()
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_rendering.py", line 330, in free
    glfw.destroy_window(self.window)
  File "/Users/n0k03zp/Desktop/miniforge3/envs/pytorch_gpu_new/lib/python3.10/site-packages/glfw/__init__.py", line 1275, in destroy_window
    window_addr = ctypes.cast(ctypes.pointer(window),
TypeError: _type_ must have storage info


 score @ 1000 = -0.34106474172898965

In [7]:
a2c_agent.training(envs, tmax, tmax, episode)

 Episode : 100	 Average reward in last 100 episode : -328.80 epsilon_policy : 0.199 epsilon_value : 4.975 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 200	 Average reward in last 100 episode : -311.16 epsilon_policy : 0.198 epsilon_value : 4.950 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 300	 Average reward in last 100 episode : -309.81 epsilon_policy : 0.197 epsilon_value : 4.926 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 400	 Average reward in last 100 episode : -309.39 epsilon_policy : 0.196 epsilon_value : 4.901 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 500	 Average reward in last 100 episode : -308.35 epsilon_policy : 0.195 epsilon_value : 4.877 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 600	 Average reward in last 100 episode : -307.80 epsilon_policy : 0.194 epsilon_value : 4.852 lambda_gae : 0.800 gamma : 0.981 entropy_reg : 0.010
 Episode : 700	 Average reward in last 100 epi

[-296.9234784709758,
 -400.46394295433913,
 -400.7080396022728,
 -317.4329907809552,
 -328.96252517209325,
 -382.2226581908077,
 -374.15204912713597,
 -368.42989119772454,
 -387.2265764207591,
 -384.4576092382939,
 -387.7910533662056,
 -386.3246051962454,
 -373.4410190527371,
 -356.0001653344516,
 -332.359945532805,
 -331.7752479094123,
 -329.34951808060515,
 -333.2104122910326,
 -351.07067118259096,
 -351.5305428349222,
 -348.18957020652147,
 -350.65704657666606,
 -351.3596865200628,
 -351.8630747296008,
 -349.94971864829415,
 -348.34686611433153,
 -340.3286220052238,
 -342.2053345367949,
 -336.11269859771085,
 -335.81904611192977,
 -335.4284082456419,
 -331.19593462939645,
 -329.08247412793105,
 -329.6506847902705,
 -329.9929406307623,
 -328.3445604885002,
 -332.1970695863291,
 -328.8525833854061,
 -319.0317491356744,
 -315.5063592897976,
 -327.08381705764384,
 -329.7390571969096,
 -326.07652115437395,
 -324.0821753178042,
 -326.041206156529,
 -336.6165765570181,
 -325.50939432817876

In [8]:
a2c_agent.training(envs, tmax, tmax, episode)

 Episode : 100	 Average reward in last 100 episode : -415.18 epsilon_policy : 0.199 epsilon_value : 4.975 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 200	 Average reward in last 100 episode : -381.43 epsilon_policy : 0.198 epsilon_value : 4.950 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 300	 Average reward in last 100 episode : -375.86 epsilon_policy : 0.197 epsilon_value : 4.926 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 400	 Average reward in last 100 episode : -361.22 epsilon_policy : 0.196 epsilon_value : 4.901 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 500	 Average reward in last 100 episode : -354.39 epsilon_policy : 0.195 epsilon_value : 4.877 lambda_gae : 0.800 gamma : 0.980 entropy_reg : 0.010
 Episode : 600	 Average reward in last 100 episode : -361.15 epsilon_policy : 0.194 epsilon_value : 4.852 lambda_gae : 0.800 gamma : 0.981 entropy_reg : 0.010
 Episode : 700	 Average reward in last 100 epi

[-249.23601440553472,
 -346.61331909803374,
 -384.52839997658987,
 -394.9442950660782,
 -398.5608627236187,
 -410.71615876500357,
 -408.81551900602983,
 -400.9584474273284,
 -395.6041600477537,
 -382.85615387078167,
 -376.4305070115186,
 -375.6380417938102,
 -374.5426940627618,
 -368.6592143404672,
 -368.8749403454509,
 -378.25072836801894,
 -376.8688790860177,
 -368.62277124496666,
 -369.6554778833573,
 -371.5313830779135,
 -380.6668401167014,
 -376.8169394265509,
 -373.5910840793752,
 -381.1387118129261,
 -381.82639022770275,
 -384.7544461139568,
 -386.4354461418528,
 -392.06109331459527,
 -393.56833526867933,
 -393.1152001050818,
 -388.85223916037955,
 -391.34890089586366,
 -400.94471706936605,
 -395.73412402241746,
 -397.48934635077137,
 -397.2787859421688,
 -398.7944025657371,
 -401.52388691454837,
 -409.3779402137136,
 -400.8518818599865,
 -398.5519820355765,
 -396.0846799322952,
 -408.0313624906463,
 -396.9063571213031,
 -395.68614150124125,
 -406.31789240283683,
 -399.528744318

In [ ]:
a2c_agent.training(envs, tmax, tmax, episode)

In [ ]:
help(envs.action_space)

In [ ]:
envs.__dict__

In [ ]:
import numpy as np
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

t = 0
_ = envs.reset()
fig, ax = plt.subplots(1,3, figsize = (30,10))
while t < 1000:
    actions = np.array([1, 0, 1])
    observations, rewards, termination, truncation, infos = envs.step(envs.action_space.sample())
    t += 1
    
    clear_output(wait=True)
    for i in range(3):
        ax[i].imshow(envs.render())
    plt.show()
    
    if np.any(termination):
        break

In [ ]:
tmp = envs.reset()

In [ ]:
tmp[0].shape

In [ ]:
help(envs)

In [ ]:
help(gym.vector.make)

In [ ]:
envs.action_space.sample().shape

In [ ]:
import numpy as np
t = 0
while t < 1000:
    actions = np.array([1, 0, 1])
    observations, rewards, termination, truncation, infos = envs.step(envs.action_space.sample())
    t += 1
    print(rewards.mean())
    if np.any(termination):
        break